# BAF627 HW1 

20249433 MFE 최재필


In [68]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

In [69]:
msf = pd.read_csv('msf.csv')
msp500_rf = pd.read_csv('msp500_risk_free.csv')

In [70]:
msf['mdate'] = pd.to_datetime(msf['mdate'])

In [71]:
msp500_rf['mdate'] = pd.to_datetime(msp500_rf['mdate'])

In [72]:
msf_ret = msf[['mdate', 'ticker', 'ret']].pivot(index='mdate', columns='ticker', values='ret')
msf_ret.tail()

ticker,AAPL,ABT,ADBE,ADI,ADP,ADSK,AIG,AMAT,AMD,AMGN,...,TMO,TXN,UNH,UNP,USB,VZ,WFC,WM,WMT,XOM
mdate,,,,,,,,,,,,,,,,,,,,,
2021-08-31,0.042438,0.044553,0.067677,-0.022578,-0.002815,-0.034379,0.152270,-0.032588,0.042659,-0.058996,...,0.027666,0.001521,0.009825,-0.003886,0.033309,-0.013983,-0.000871,0.046203,0.042792,-0.037867
2021-09-30,-0.068037,-0.065205,-0.132560,0.027800,-0.039179,-0.080364,0.011913,-0.047362,-0.070629,-0.057110,...,0.029985,0.006810,-0.057847,-0.096062,0.043736,-0.018000,0.015536,-0.033363,-0.058879,0.078870
2021-10-31,0.058657,0.094895,0.129646,0.035885,0.122899,0.113757,0.076517,0.061524,0.168416,-0.026711,...,0.108064,-0.018626,0.178456,0.231570,0.015646,-0.007036,0.102349,0.072777,0.072033,0.096056
2021-11-30,0.104940,-0.024207,0.029968,0.038965,0.028509,-0.199679,-0.109832,0.078888,0.317225,-0.030584,...,-0.000379,0.026083,-0.035290,-0.023861,-0.083320,-0.051330,-0.062158,0.002746,-0.058827,-0.058167
2021-12-31,0.074229,0.119027,-0.153452,-0.021026,0.072459,0.106220,0.087072,0.069094,-0.091368,0.131185,...,0.054786,-0.020273,0.133650,0.074138,0.023310,0.033618,0.004186,0.042354,0.032781,0.022560


In [73]:
sp500_s = msp500_rf[['mdate', 'spindx']].set_index('mdate')
sp500_ret = sp500_s.pct_change().dropna()

In [74]:
rf_s = msp500_rf[['mdate', 'rf']].set_index('mdate')

## 1. mean & covariance

- Assume there is no risk-free asset
- Short-selling is allowed

In [75]:
cov_df = msf_ret.cov()
mean_s = msf_ret.mean()

In [76]:
std_s = msf_ret.std()

In [77]:
cov_2d = cov_df.values
mean_v = mean_s.values
std_v = std_s.values

### (a) 

- Derive the mean-variance frontier using the standard deviation for measuring risk
- Plot the mean-variance frontier
- Indicate the global minimum portfolio (GMVP) on the plot

In [78]:
def port_mean(W, mean_v):
    """Get the mean of the portfolio

    Args:
        W (np.ndarray): 1*n array of weights
        mean_v (np.ndarray): 1*n array of mean returns

    Returns:
        float: weighted mean return of the portfolio
    """
    return np.sum(W * mean_v)

In [79]:
n = len(mean_v)
W = np.ones((1, n)) / n

In [80]:
port_mean(W, mean_v) 

0.014625304555427415

In [81]:
def port_var(W, cov_2d):
    """Get the variance of the portfolio

    Args:
        W (np.ndarray): 1*n array of weights
        cov_2d (np.ndarray): n*n array of covariance matrix

    Returns:
        float: variance of the portfolio
    """    
    return np.dot(W, np.dot(cov_2d, W.T))[0][0]

In [82]:
port_var(W, cov_2d)

0.001940290536648992

In [83]:
def port_sharpe(W, mean_v, cov_2d, rf):
    """Get the Sharpe ratio of the portfolio

    Args:
        W (np.ndarray): 1*n array of weights
        mean_v (np.ndarray): 1*n array of mean returns
        cov_2d (np.ndarray): n*n array of covariance matrix
        rf (float): risk-free rate

    Returns:
        float: Sharpe ratio of the portfolio
    """    
    return ( port_mean(W, mean_v) - rf ) / np.sqrt(port_var(W, cov_2d))

In [ ]:
import matplotlib.pyplot as plt
import scipy.optimize as sco

def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.dot(weights, mean_returns)
    risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, risk

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0):
    p_returns, p_risk = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(p_returns - risk_free_rate) / p_risk

def optimize_portfolio(mean_returns, cov_matrix, risk_free_rate=0):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((-1, 1) for asset in range(num_assets))
    
    result = sco.minimize(negative_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                          method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def efficient_frontier(mean_returns, cov_matrix, num_portfolios=100):
    results = np.zeros((3, num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.uniform(-1, 1, len(mean_returns))
        weights /= np.sum(np.abs(weights))
        p_returns, p_risk = portfolio_performance(weights, mean_returns, cov_matrix)
        results[0,i] = p_risk
        results[1,i] = p_returns
        results[2,i] = results[1,i] / results[0,i]
        weights_record.append(weights)
    
    return results, weights_record

# Generate the efficient frontier
num_portfolios = 10000
results, weights = efficient_frontier(mean_returns, cov_matrix, num_portfolios)

# Find the optimal portfolio (maximum Sharpe ratio)
max_sharpe_idx = np.argmax(results[2])
sdp, rp = results[0, max_sharpe_idx], results[1, max_sharpe_idx]
max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx], index=returns_df.columns, columns=['allocation'])
max_sharpe_allocation.allocation = [round(i*100,2) for i in max_sharpe_allocation.allocation]

# Plot the efficient frontier
plt.figure(figsize=(10, 7))
plt.scatter(results[0,:], results[1,:], c=results[2,:], cmap='YlGnBu', marker='o')
plt.colorbar(label='Sharpe ratio')
plt.scatter(sdp, rp, marker='*', color='r', s=200, label='Max Sharpe ratio')
plt.title('Efficient Frontier with Short Selling Allowed')
plt.xlabel('Risk (Std. Deviation)')
plt.ylabel('Return')
plt.legend(labelspacing=0.8)
plt.show()
